In [3]:
!pip install cirq


In [4]:
import cirq
import numpy as np
from scipy.optimize import minimize


# Define the single-qubit parametric circuit U3
def U3_gate(theta, phi, lam):
    return cirq.Circuit(
        cirq.rz(theta).on(cirq.LineQubit(0)),
        cirq.ry(phi).on(cirq.LineQubit(0)),
        cirq.rz(lam).on(cirq.LineQubit(0))
    )


# Define the Hamiltonians
I = cirq.PauliString(cirq.I(cirq.LineQubit(0)))
X = cirq.PauliString(cirq.X(cirq.LineQubit(0)))
Y = cirq.PauliString(cirq.Y(cirq.LineQubit(0)))
Z = cirq.PauliString(cirq.Z(cirq.LineQubit(0)))


# Function to calculate expectation value
def expectation_value(circuit, hamiltonian, repetitions=1000):
    simulator = cirq.Simulator()
    avg = simulator.simulate_expectation_values(circuit, observables=[hamiltonian])
    return avg[0].real


# Function to compute gradients using the parameter shift rule
def compute_gradients(circuit, hamiltonian, param_values, shift=np.pi / 2):
    gradients = []

    for i in range(len(param_values)):
        params_pos = param_values.copy()
        params_pos[i] += shift
        circuit_pos = U3_gate(*params_pos)
        expectation_pos = expectation_value(circuit_pos, hamiltonian)

        params_neg = param_values.copy()
        params_neg[i] -= shift
        circuit_neg = U3_gate(*params_neg)
        expectation_neg = expectation_value(circuit_neg, hamiltonian)

        gradient = (expectation_pos - expectation_neg) / (2 * np.sin(shift))
        gradients.append(gradient)

    return np.array(gradients)


def grid_search(hamiltonian):
    optimal_params = None
    optimal_expectation = float('inf')

    for theta in np.linspace(0, 2 * np.pi, 10):
        for phi in np.linspace(0, 2 * np.pi, 10):
            for lam in np.linspace(0, 2 * np.pi, 10):
                circuit = U3_gate(theta, phi, lam)
                expectation = expectation_value(circuit, hamiltonian)

                if expectation < optimal_expectation:
                    optimal_expectation = expectation
                    optimal_params = (theta, phi, lam)

    return optimal_params, optimal_expectation


# Gradient-based optimization using scipy's minimize
def optimize_parameters(hamiltonian, initial_params):
    result = minimize(
        fun=lambda params: expectation_value(U3_gate(*params), hamiltonian),
        x0=initial_params,
        jac=lambda params: compute_gradients(U3_gate(*params), hamiltonian, params),
        method='L-BFGS-B'
    )
    optimal_params = result.x
    optimal_expectation = result.fun
    return optimal_params, optimal_expectation


# Test for each Hamiltonian
hamiltonians = [I, X, Y, Z]
for hamiltonian in hamiltonians:
    # Brute force grid search
    optimal_params_grid, optimal_expectation_grid = grid_search(hamiltonian)
    print(f"Optimal parameters (Grid Search) for {hamiltonian} : {optimal_params_grid}")
    print(f"Optimal expectation value (Grid Search): {optimal_expectation_grid}\n")

    # Gradient-based optimization
    initial_params = np.random.rand(3) * 2 * np.pi  # or initialize differently
    optimal_params_grad, optimal_expectation_grad = optimize_parameters(hamiltonian, initial_params)
    print(f"Optimal parameters (Gradient Descent) for {hamiltonian} : {optimal_params_grad}")
    print(f"Optimal expectation value (Gradient Descent): {optimal_expectation_grad}\n")



Optimal parameters (Grid Search) for I : (0.6981317007977318, 2.792526803190927, 2.792526803190927)
Optimal expectation value (Grid Search): 0.9999997615814209

Optimal parameters (Gradient Descent) for I : [1.97395345 6.15122782 1.34478969]
Optimal expectation value (Gradient Descent): 0.9999999403953552

Optimal parameters (Grid Search) for X(q(0)) : (1.3962634015954636, 4.886921905584122, 0.0)
Optimal expectation value (Grid Search): -0.9848078489303589

Optimal parameters (Gradient Descent) for X(q(0)) : [ 0.64746059  1.5706423  -3.14169589]
Optimal expectation value (Gradient Descent): -1.0

Optimal parameters (Grid Search) for Y(q(0)) : (0.6981317007977318, 1.3962634015954636, 4.886921905584122)
Optimal expectation value (Grid Search): -0.9698464870452881

Optimal parameters (Gradient Descent) for Y(q(0)) : [4.43891769 4.7123887  1.57079684]
Optimal expectation value (Gradient Descent): -1.0000001192092896

Optimal parameters (Grid Search) for Z(q(0)) : (1.3962634015954636, 2.792